In [3]:
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2Tokenizer, GPT2LMHeadModel

In [5]:
mdf = pd.read_excel('./Dataset/FeasibilityQA_dataset_MCQ.xlsx')
bdf = pd.read_excel('./Dataset/FeasibilityQA_dataset_BCQ.xlsx')

In [6]:
mdf.head()

,category,subcategory,premise,knowledge,question_text,options,correct_answers,correct_option
0,attribute_comparison,account_balance,Charlie has 622 dollars in his account. He wan...,One can not withdraw more money than the amoun...,At what cost could Charlie buy the vehicle fro...,"[{'A': 550}, {'B': 350}, {'C': 650}, {'D': 750...","[550, 350]","['A', 'B']"
1,attribute_comparison,account_balance,Jackson wants to buy a sports car. He has 534 ...,One can not withdraw more money than the amoun...,At what cost could Jackson buy the vehicle fro...,"[{'A': 550}, {'B': 400}, {'C': 500}, {'D': 650}]","[400, 500]","['B', 'C']"
2,attribute_comparison,account_balance,Cost of a bike is 692 dollars. William wants t...,One can not withdraw more money than the amoun...,How much money William could have to buy the v...,"[{'A': 500}, {'B': 600}, {'C': 800}, {'D': 450...",[800],['C']
3,attribute_comparison,account_balance,Cost of a bike is 552 dollars. Nicolas wants t...,One can not withdraw more money than the amoun...,How much money Nicolas could have to buy the v...,"[{'A': 750}, {'B': 500}, {'C': 450}, {'D': 550...",[750],['A']
4,attribute_comparison,age,Cole is 72 years old.,Son's age would be less than that of the father,How old could Cole's son be?,"[{'A': 47}, {'B': 22}, {'C': 97}, {'D': 72}]","[47, 22]","['A', 'B']"


In [7]:
bdf.head()

,category,subcategory,knowledge,premise,hypothesis,binary_classification_label
0,attribute_comparison,account_balance,One can not withdraw more money than the amoun...,Charlie has 622 dollars in his account. He wan...,The cost of the vehicle is 500. Charlie could ...,True
1,attribute_comparison,account_balance,One can not withdraw more money than the amoun...,Charlie has 622 dollars in his account. He wan...,The cost of the vehicle is 800. Charlie could ...,False
2,attribute_comparison,account_balance,One can not withdraw more money than the amoun...,Jackson wants to buy a sports car. He has 534 ...,The cost of the vehicle is 500. Jackson could ...,True
3,attribute_comparison,account_balance,One can not withdraw more money than the amoun...,Jackson wants to buy a sports car. He has 534 ...,The cost of the vehicle is 700. Jackson could ...,False
4,attribute_comparison,account_balance,One can not withdraw more money than the amoun...,Cost of a bike is 692 dollars. William wants t...,William has 700 dollars in his account. Willia...,True


In [8]:
#Question for binary question Zero Shot
question_binary = 'Validate the hypothesis as True or False based on Knowledge and Premise:'

#Learning samples for binary questions One & Few Shot
examples_list_binary = [
                    {'knowledge': "One can not withdraw more money than the amount in their bank account.",
                    'premise': "Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.",
                    'hypothesis': "Charlie could buy the vehicle from his account", 
                    'result': True
                    }, 
                    {'knowledge': "Resistance is inversely proportional to the area of the metalic wire.",
                    'premise': "Victor's experiment is to observe the relationship between resistance and area using two similar wires of different areas. The area of the wire with larger resistance is 32 units.",
                    'hypothesis': "Area of the other wire could be 25 units.", 
                    'result': False
                    }, 
                    {'knowledge': "Legal age to vote is 18 years in India.",
                    'premise': "Evan is a citizen of India and he recently cast his first vote in the national election.",
                    'hypothesis': "Evan is 21 years old when he casted the vote.", 
                    'result': True
                    }, 
                    {'knowledge': "To pass an exam, you need to get at least passing marks.",
                    'premise': "Anthony passed an exam where the passing score was 77.",
                    'hypothesis': "Anthony could have scored 76 marks in that exam.", 
                    'result': False
                    }
                ]

#Question for MCQ questions Zero Shot
question_mcq = 'Choose the correct option(s) based on Knowledge (if given) and Premise:'

#Learning samples for MCQ questions One & Few Shot
examples_list_mcq = [
                    {'knowledge': "One can not withdraw more money than the amount in their bank account.",
                    'premise': "Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.",
                    'question': "At what cost could Charlie buy the vehicle from his account?",
                     'options': [{'A': 550}, {'B': 350}, {'C': 650}, {'D': 750}, {'E': 'None'}], 
                    'result': [550, 350]
                    }, 
                    {'knowledge': "Smaller planet has a smaller radius in comparison to a larger planet.",
                    'premise': "Planet A is bigger in radius that planet B. The radius of planet A is 43 units.",
                    'question': "What could be the radius of planet B?",
                     'options': [{'A': 46}, {'B': 39}, {'C': 43}, {'D': 41}],
                    'result': [39, 41]
                    }, 
                    {'knowledge': "Potential energy is directly proportional to the height.",
                    'premise': "Antonio's experiment is to observe the relationship between potential energy and height of two identical balls. The potential energy of the ball at lower height is 29 units.",
                    'question': "What could be the potential energy of the other ball?",
                     'options': [{'A': 31}, {'B': 28}, {'C': 33}, {'D': 24}], 
                    'result': [31, 33]
                    }, 
                    {'knowledge': "Speed of the upstream river reduces the speed of the raft.",
                    'premise': "Samuel participates in a boat racing competition. He first rows the boat in still water with a speed of 20 and then starts to go upstream.",
                    'question': "What could be the overall speed of the raft in upstream?",
                     'options': [{'A': 24}, {'B': 15}, {'C': 16}, {'D': 19}], 
                    'result': [15, 16, 19]
                    }
                ]

In [9]:
#Get prompts in required format
def create_prompts(val, question_type, examples = None, question_zs = None, knowledge = True, premise = True, shot = 'zero'):
    if shot == 'zero':
        prompt_str = question_zs + "\n"
        if knowledge:
            knowledge_var = f"Knowledge: {val['knowledge']}\n"
            prompt_str += knowledge_var 
        if premise:
            premise_var = f"Premise: {val['premise']}\n"
            prompt_str += premise_var

        if question_type == 'binary':
            prompt_str += f"Hypothesis: {val['hypothesis']}\nResult:"
        if question_type == 'mcq':
            prompt_str += f"Question: {val['question_text']}\n"
            prompt_str += f"Options: {val['options']}\nResult:"

        return prompt_str

    elif shot == 'one':
        try:
            prompt_str = question_zs + "\n"
            for item, text in examples[1].items():
                if knowledge and item  == 'knowledge':
                    knowledge_var = f"Knowledge: {text}\n"
                    prompt_str += knowledge_var 
                if premise and item  == 'premise':
                    premise_var = f"Premise: {text}\n"
                    prompt_str += premise_var
                if item == 'hypothesis':
                    prompt_str += f"Hypothesis: {text}\n"
                if item == 'question':
                    prompt_str += f"Question: {text}\n"
                if item == 'options':
                    prompt_str += f"Options: {text}\n"
                if item == 'result': 
                    prompt_str += f"Result: {text}\n\n" 
        except:
            raise Exception("Provide atleast one example")

        if knowledge:
            knowledge_var = f"Knowledge: {val['knowledge']}\n"
            prompt_str += knowledge_var 
        if premise:
            premise_var = f"Premise: {val['premise']}\n"
            prompt_str += premise_var
        
        if question_type == 'binary':
            prompt_str += f"Hypothesis: {val['hypothesis']}\nResult:"
        if question_type == 'mcq':
            prompt_str += f"Question: {val['question_text']}\n"
            prompt_str += f"Hypothesis: {val['options']}\nResult:"
        return prompt_str
    
    elif shot == 'few':
        try:
            prompt_str = question_zs + "\n"
            for example in examples:
                for item, text in example.items():
                    if knowledge and item  == 'knowledge':
                        knowledge_var = f"Knowledge: {text}\n"
                        prompt_str += knowledge_var 
                    if premise and item  == 'premise':
                        premise_var = f"Premise: {text}\n"
                        prompt_str += premise_var
                    if item == 'hypothesis':
                        prompt_str += f"Hypothesis: {text}\n"
                    if item == 'question':
                        prompt_str += f"Question: {text}\n"
                    if item == 'options':
                        prompt_str += f"Options: {text}\n"
                    if item == 'result': 
                        prompt_str += f"Result: {text}\n\n" 
        except:
            raise Exception("Provide atleast one example")

        if knowledge:
            knowledge_var = f"Knowledge: {val['knowledge']}\n"
            prompt_str += knowledge_var 
        if premise:
            premise_var = f"Premise: {val['premise']}\n"
            prompt_str += premise_var
        
        if question_type == 'binary':
            prompt_str += f"Hypothesis: {val['hypothesis']}\nResult:"
        if question_type == 'mcq':
            prompt_str += f"Question: {val['question_text']}\n"
            prompt_str += f"Hypothesis: {val['options']}\nResult:"
        return prompt_str
    
    else:
        raise Exception("Not a valid entry for shot argument. It can take values which are: zero, one or few")


#Get response from Transformers
def hf_response(df, output_path, tokenizer, model, pred_col_name = 'GPT3Pred', start_idx = 0, end_idx = None, knowledge = True, 
                premise = True, shot = 'one', dump = True, question_type = 'binary', sample_size = None):
    
    print('Data size: ', df.shape[0])
    
    if isinstance(sample_size, float):
        df = df.sample(frac = sample_size).reset_index().reset_index(drop = True)
        print('Data size after resampling: ', df.shape[0])
    elif isinstance(sample_size, int):
        df = df.sample(n = sample_size).reset_index(drop = True)
        print('Data size after resampling: ', df.shape[0])
    else:
        pass
    
    #If new file create new column
    if start_idx == 0:
        df[pred_col_name] = None

    if end_idx is None:
        end_idx = len(df)
    
    for idx, row in enumerate(df.iloc[start_idx:end_idx].iterrows(), start_idx):
        print('Row Number: ', idx+1)
        if question_type == 'binary':
            prompt_str = create_prompts(row[1], question_type, examples = examples_list_binary, question_zs = question_binary, 
                                        knowledge = knowledge, premise = premise, shot = shot)
        if question_type == 'mcq':
            prompt_str = create_prompts(row[1], question_type, examples = examples_list_mcq, question_zs = question_mcq, 
                                        knowledge = knowledge, premise = premise, shot = shot)
        try:
            input_ids = tokenizer(prompt_str, return_tensors="pt").input_ids
            outputs = model.generate(input_ids, max_new_tokens=30)
            result = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(prompt_str + ' ' + result)
            df.loc[idx, pred_col_name] = result
            print('\n**********************\n')
            
            if dump:
                df.to_csv(output_path, index = False)
        except:
            print('Resources exhausted probably!!!')
            break

    if not dump:
        return df

In [10]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

# BCQ

In [ ]:
print('\n **** ZERO SHOT ****\n')
#Zero Shot Results - With Knowledge
outpath = './Binary/ZeroShot_WK_BCQ_Output.csv'
zsbf1 = bdf.copy()
hf_response(zsbf1, outpath, tokenizer, model, pred_col_name='GPT2Pred', knowledge = True, shot = 'zero', dump = True, question_type = 'binary')

#Zero Shot Results - Without Knowledge
outpath = './GPT2/Binary/ZeroShot_WOK_BCQ_Output.csv'
zsbf2 = bdf.copy()
hf_response(zsbf2, outpath, tokenizer, model, pred_col_name='GPT2Pred', knowledge = False, shot = 'zero', dump = True, question_type = 'binary')


print('\n **** ONE SHOT ****\n')


#One Shot Results - With Knowledge
outpath = './Binary/OneShot_WK_BCQ_Output.csv'
osbf1 = bdf.copy()
hf_response(osbf1, outpath, tokenizer, model, pred_col_name='GPT2Pred', knowledge = True, shot = 'one', dump = True, question_type = 'binary')


#One Shot Results - Without Knowledge
outpath = './Binary/OneShot_WOK_BCQ_Output.csv'
osbf2 = bdf.copy()
hf_response(osbf1, outpath, tokenizer, model, pred_col_name='GPT2Pred', knowledge = False, shot = 'one', dump = True, question_type = 'binary')


print('\n **** FEW SHOT ****\n')


#Few Shot Results - With Knowledge
outpath = './Binary/FewShot_WK_BCQ_Output.csv'
fsbf1 = bdf.copy()
hf_response(fsbf1, outpath, tokenizer, model, pred_col_name='GPT2Pred', knowledge = True, shot = 'few', dump = True, question_type = 'binary')


#Few Shot Results - Without Knowledge
outpath = './Binary/FewShot_WOK_BCQ_Output.csv'
fsbf2 = bdf.copy()
hf_response(fsbf2, outpath, tokenizer, model, pred_col_name='GPT2Pred', knowledge = False, shot = 'few', dump = True, question_type = 'binary')

Output hidden; open in https://colab.research.google.com to view.

# MCQ

In [ ]:
print('\n **** ZERO SHOT RESULTS **** \n')

#Zero Shot Results - With Knowledge
outpath = './MCQ/ZeroShot_WK_MCQ_Output.csv'
zsmf1 = mdf.copy()
hf_response(zsmf1, outpath, tokenizer, model, pred_col_name='GPT2Pred', knowledge = True, shot = 'zero', dump = True, question_type = 'mcq')

#Zero Shot Results - Without Knowledge
outpath = './MCQ/ZeroShot_WOK_MCQ_Output.csv'
zsmf2 = mdf.copy()
hf_response(zsmf2, outpath, tokenizer, model, pred_col_name='GPT2Pred', knowledge = False, shot = 'zero', dump = True, question_type = 'mcq')


print('\n **** ONE SHOT RESULTS **** \n')


#One Shot Results - With Knowledge
outpath = './MCQ/OneShot_WK_MCQ_Output.csv'
osmf1 = mdf.copy()
hf_response(osmf1, outpath, tokenizer, model, pred_col_name='GPT2Pred', knowledge = True, shot = 'one', dump = True, question_type = 'mcq')


#One Shot Results - With Knowledge
outpath = './MCQ/OneShot_WOK_MCQ_Output.csv'
osmf2 = mdf.copy()
hf_response(osmf2, outpath, tokenizer, model, pred_col_name='GPT2Pred', knowledge = False, shot = 'one', dump = True, question_type = 'mcq')


print('\n **** FEW SHOT RESULTS **** \n')


#Few Shot Results - With Knowledge
outpath = './MCQ/FewShot_WK_MCQ_Output.csv'
fsmf1 = mdf.copy()
hf_response(fsmf1, outpath, tokenizer, model, pred_col_name='GPT2Pred', knowledge = True, shot = 'few', dump = True, question_type = 'mcq')


#Few Shot Results - With Knowledge
outpath = './MCQ/FewShot_WOK_MCQ_Output.csv'
fsmf2 = mdf.copy()
hf_response(fsmf2, outpath, tokenizer, model, pred_col_name='GPT2Pred', knowledge = False, shot = 'few', dump = True, question_type = 'mcq')

Output hidden; open in https://colab.research.google.com to view.